<a href="https://colab.research.google.com/github/Dr-Carlos-Villasenor/Clase_Aprendizaje_Profundo/blob/main/L08_Transferencia_de_Aprendizaje.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transferencia de aprendizaje
## Dr. Carlos Villaseñor

Importar bibliotecas



In [ ]:
import pandas as pd
import numpy as np
import os
import keras
import matplotlib.pyplot as plt
from keras.layers import Dense,GlobalAveragePooling2D
from keras.applications import MobileNet
from keras.preprocessing import image
from keras.applications.mobilenet import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.optimizers import Adam

Importar MobileNet base pero descartaremos las últimas capa

In [ ]:
base_model=MobileNet(weights='imagenet',include_top=False)

Crear nueva Red Neuronal

In [ ]:
x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(256,activation='relu')(x)
x=Dense(64,activation='relu')(x)
preds=Dense(3, activation='softmax')(x)

Pegar Redes Neuronales

In [ ]:
model=Model(inputs=base_model.input, outputs=preds)

Definir que capas se entrenan

In [ ]:
for layer in model.layers[:-5]:
    layer.trainable=False
for layer in model.layers[-5:]:
    layer.trainable=True

Cargar datos nuevos

- Carge el archivo 'train.zip' en la pestaña archivos en el lateral izquierdo
- Corra la siguiente linea para descomprimir los archivos

In [ ]:
!wget "https://github.com/Dr-Carlos-Villasenor/Taller_CNN/raw/main/train.zip"

In [ ]:
!unzip train.zip

Crear generador desde carpeta

In [ ]:
train_datagen=ImageDataGenerator(zoom_range=0.2, horizontal_flip=True,
                                 width_shift_range=0.2,
                                 height_shift_range=0.2,
                                 preprocessing_function=preprocess_input)

train_generator=train_datagen.flow_from_directory('./train/',
                                                 target_size=(224,224),
                                                 color_mode='rgb',
                                                 batch_size=32,
                                                 class_mode='categorical',
                                                 shuffle=True)



Compilar Modelo y entrenar desde carpeta

In [ ]:
model.compile(optimizer='Adam',loss='categorical_crossentropy',
              metrics=['accuracy'])

step_size_train=train_generator.n//train_generator.batch_size

model.fit(train_generator,
          steps_per_epoch=step_size_train,
          epochs=6)